In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import helper_functions

EPOCHS = 40
BATCH_SIZE = 32
PATIENCE = 5
LEARNING_RATE = 0.0001

In [2]:
# load dataset
train_data = helper_functions.load_data()

X = np.array(train_data['mfccs'], dtype=float)
y = np.array([helper_functions.one_hot_to_label(label) for label in train_data['classes']])
y_train_one_hot = np.array(train_data['classes'])
y = y_train_one_hot.argmax(axis=1)
print(y)

[11 11 11 ... 11 11 11]


In [3]:
# create train/validation/test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1)

# convert inputs which are 2 dimensional MFCC arrays into 3 dimensional arrays
# Currently shape of the arrays is (# segments, 13), we want to add a 3rd dimension so we have (# segments, 13, 1)
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

n_dim = X_train.shape[2]

print(f'x_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'n_dim: {n_dim}')

x_train shape: (52423, 15, 173, 1)
y_train shape: (52423,)
n_dim: 173


In [4]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

# build network architecture using convolutional layers
model = tf.keras.models.Sequential()

# 1st conv layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

# 2nd conv layer
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                                     kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))

# 3rd conv layer
model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2), padding='same'))

# flatten output and feed into dense layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
tf.keras.layers.Dropout(0.3)

# softmax output layer
model.add(tf.keras.layers.Dense(35, activation='softmax'))

optimiser = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

# compile model
model.compile(optimizer=optimiser, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# print model parameters on console
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 171, 64)       640       
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 171, 64)       256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 86, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 84, 32)         18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 84, 32)         128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 42, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 41, 32)         4

In [5]:
# train the model
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_validation, y_validation))

Epoch 1/40
1639/1639 [==============================] - 486s 296ms/step - loss: 1.8855 - accuracy: 0.5733 - val_loss: 1.0270 - val_accuracy: 0.7008
Epoch 2/40
1639/1639 [==============================] - 206s 126ms/step - loss: 0.9337 - accuracy: 0.7296 - val_loss: 0.7441 - val_accuracy: 0.7758
Epoch 3/40
1639/1639 [==============================] - 175s 107ms/step - loss: 0.6893 - accuracy: 0.8038 - val_loss: 0.6241 - val_accuracy: 0.8156
Epoch 4/40
1639/1639 [==============================] - 197s 120ms/step - loss: 0.5341 - accuracy: 0.8538 - val_loss: 0.5168 - val_accuracy: 0.8555
Epoch 5/40
1639/1639 [==============================] - 252s 154ms/step - loss: 0.4538 - accuracy: 0.8802 - val_loss: 0.4743 - val_accuracy: 0.8699
Epoch 6/40
1639/1639 [==============================] - 149s 90ms/step - loss: 0.3943 - accuracy: 0.8984 - val_loss: 0.4521 - val_accuracy: 0.8757
Epoch 7/40
1639/1639 [==============================] - 149s 91ms/step - loss: 0.3570 - accuracy: 0.9074 - val_lo

In [12]:
import sys
model.save('models/ross_cnn')

INFO:tensorflow:Assets written to: models/ross_cnn/assets
